In [2]:
#load library
import simfin as sf
from simfin.names import *

import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import seaborn as sns
import numpy as np
import datetime

In [3]:
#set API key

sf.set_api_key('poepcJpfyS4oMRuSEEytWkErpDmdsa0C')

In [4]:
#get list of 5 years - 20 quarters SP500 companies

SPlist = pd.read_csv('FullCompanyList.csv')
SPlist

,3/31/20,12/31/19,9/30/19,6/30/19,3/31/19,12/31/18,9/30/18,6/30/18,3/31/18,12/31/17,...,9/30/12,6/30/12,3/31/12,12/31/11,9/30/11,6/30/11,3/31/11,12/31/10,9/30/10,6/30/10
0,A,A,A,A,A,A,A,A,A,A,...,A,A,A,A,A,A,A,A,A,A
1,AAL,AAL,AAL,AAL,AAL,AAL,AAL,AAL,AAL,AAL,...,AA,AA,AA,AA,AA,AA,AA,AA,AA,AA
2,AAP,AAP,AAP,AAP,AAP,AAP,AAP,AAP,AAP,AAP,...,AAPL,AAPL,AAPL,AAPL,AAPL,AAPL,AAPL,AAPL,AAPL,AAPL
3,AAPL,AAPL,AAPL,AAPL,AAPL,AAPL,AAPL,AAPL,AAPL,AAPL,...,ABC,ABC,ABC,ABC,ABC,ABC,ABC,ABC,ABC,ABC
4,ABBV,ABBV,ABBV,ABBV,ABBV,ABBV,ABBV,ABBV,ABBV,ABBV,...,ABT,ABT,ABT,ABT,ABT,ABT,ABT,ABT,ABT,ABT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,YUM,XYL,XYL,XYL,XYL,XYL,XYL,XYL,XYL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
501,ZBH,ZBH,YUM,YUM,YUM,YUM,YUM,YUM,YUM,YUM,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502,ZBRA,ZBRA,ZBH,ZBH,ZBH,ZBH,ZBH,ZBH,ZBH,ZBH,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
503,ZION,ZION,ZION,ZION,ZION,ZION,ZION,ZION,ZION,ZION,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
#Convert dataframe to a dictionary

listDict = SPlist.to_dict('list')

In [6]:
#Get a list of values from dict

values = [*listDict.values()]

In [7]:
#Get a list of column names from dict

periodList  =[*listDict]

In [8]:
#Convert columns into quarters

df = pd.DataFrame({'Period': pd.to_datetime(periodList)})
df['Period'] = df['Period'].dt.to_period('Q')
newkey = df['Period'].astype(str).to_list()

In [9]:
#Use new column name and old values to create new dictionary

newList = dict(zip(newkey, values)) 

In [10]:
#Load data from API

sf.set_data_dir('~/simfin_data/')

df = sf.load_income(variant='quarterly', market='us')
dfb = sf.load_balance(variant='quarterly-full', market='us')
dfp = sf.load_shareprices(variant='daily', market='us')

Dataset "us-income-quarterly" on disk (16 days old).
- Loading from disk ... Done!
Dataset "us-balance-quarterly-full" on disk (16 days old).
- Loading from disk ... Done!
Dataset "us-shareprices-daily" on disk (16 days old).
- Loading from disk ... Done!


In [11]:
def get_data (tickers):
    #load data from ticker lists
    
    df_income_qtr = df.loc[newList[tickers]]
    df_balance_qtr = dfb.loc[newList[tickers]]
    df_price = dfp.loc[newList[tickers]]
    
    
    #Organize and combine data, do calculations
    df_income_qtr['NOPAT'] = df_income_qtr['Operating Income (Loss)'] - df_income_qtr['Income Tax (Expense) Benefit, Net']
    df_balance_qtr['Invested Capital'] = df_balance_qtr['Total Noncurrent Liabilities'] + df_balance_qtr['Total Equity']
    
    income = df_income_qtr[['Fiscal Year','Publish Date','NOPAT']]
    balance = df_balance_qtr[['Invested Capital']]
    income2=income.reset_index()
    balance2 = balance.reset_index()
    
    combine = pd.merge(income,balance,left_index=True, right_index=True)
    combine['ROIC'] = combine['NOPAT']/combine['Invested Capital']
    
    combine['Publish Date'] = pd.to_datetime(combine['Publish Date']).dt.strftime('%Y-%m-%d')
    combine['Year_Quarter'] = pd.to_datetime(combine['Publish Date']).dt.to_period('Q').astype(str)
    
    
    roic = combine[combine['Year_Quarter'] == tickers]
    roic = roic[roic['ROIC'].notnull()]
    roic = roic.reset_index()
    
    df_price = df_price.reset_index()
    df_price['Market Cap'] = df_price['Adj. Close'] * df_price['Shares Outstanding']
    
    tmp = df_price.groupby(['Ticker', pd.PeriodIndex(df_price['Date'], freq='Q', name='Quarter')]).last().reset_index(level = 0)
    tmp['Return'] = tmp['Adj. Close']/tmp.groupby('Ticker')['Adj. Close'].shift() - 1
    individual_df = {str(i): df_price.reset_index(drop=True) for i, df_price in tmp.groupby(['Quarter'])}    
    
    
    price = individual_df[tickers][['Ticker','Adj. Close','Market Cap','Return']]
    
    full = roic.set_index('Ticker').join(price.set_index('Ticker'))
    
    #Divide the companies into five quartiles
    x = int(5 * round(len(full)/5)-5)
    
    qua1, qua2, qua3, qua4, qua5 = np.split(full[:x].sort_values(by = 'ROIC', ascending = False), 5)
    
    #Calculate market cap for each quartile
    mktcap1 = qua1['Market Cap'].sum()
    mktcap2 = qua2['Market Cap'].sum()
    mktcap3 = qua3['Market Cap'].sum()
    mktcap4 = qua4['Market Cap'].sum()
    mktcap5 = qua5['Market Cap'].sum()
    
    #Calculate weight for each quartile
    qua1['Weight'] = qua1['Market Cap']/mktcap1
    qua2['Weight'] = qua2['Market Cap']/mktcap2
    qua3['Weight'] = qua3['Market Cap']/mktcap3
    qua4['Weight'] = qua4['Market Cap']/mktcap4
    qua5['Weight'] = qua5['Market Cap']/mktcap5
   
    #calculate the return index for each quartile
    index1 = (qua1['Return']*qua1['Weight']).sum()
    index2 = (qua2['Return']*qua2['Weight']).sum()
    index3 = (qua3['Return']*qua3['Weight']).sum()
    index4 = (qua4['Return']*qua4['Weight']).sum()
    index5 = (qua5['Return']*qua5['Weight']).sum()
    
    #get the data into a list
    List = [index1,index2,index3,index4,index5]
    
    
    return List

In [12]:
# Run all ticker list in the function above and create a list called all_index
# The data is very large so it might take some time


all_index = []
for keys in newkey:
    index = get_data(keys)
    all_index.append(index)

/Users/559/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/Users/559/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [13]:
#Create dataframe based on data get

df_index = pd.DataFrame(all_index, index=newkey, 
                                 columns=['Q1_Index','Q2_Index','Q3_Index','Q4_Index','Q5_Index'])

In [14]:
#show data

df_index

,Q1_Index,Q2_Index,Q3_Index,Q4_Index,Q5_Index
2020Q1,-0.111181,-0.145482,-0.134653,-0.170731,-0.163288
2019Q4,0.139736,0.115160,0.049101,0.064368,0.058528
2019Q3,0.020156,0.048794,0.012146,0.004329,0.031006
2019Q2,0.052012,0.044418,0.061272,0.057010,0.043061
2019Q1,0.195402,0.138060,0.160400,0.141934,0.150713
2018Q4,-0.169242,-0.100845,-0.118183,-0.102574,-0.110649
2018Q3,0.059661,0.123044,0.109890,0.047674,0.066581
2018Q2,0.077141,0.042160,0.074941,0.047744,0.029747
2018Q1,0.015145,0.014127,0.040611,0.022095,-0.046955
2017Q4,0.078768,0.087921,0.094561,0.038879,0.052035


In [15]:
ax = df_index.plt(figsize = (20,10))

AttributeError: 'DataFrame' object has no attribute 'plt'